In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv('pseudodata_præoperation.csv')

#make pd to np
X = data.to_numpy()
y = np.random.choice([0, 1], size=len(data))

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into holdout set and remaining set
X_remain, X_holdout, y_remain, y_holdout = train_test_split(X, y, test_size=500, random_state=42)

# Convert to PyTorch tensors
X_remain = torch.tensor(X_remain, dtype=torch.float32)
y_remain = torch.tensor(y_remain, dtype=torch.float32).unsqueeze(1)
X_holdout = torch.tensor(X_holdout, dtype=torch.float32)
y_holdout = torch.tensor(y_holdout, dtype=torch.float32).unsqueeze(1)


In [7]:
def train_and_evaluate_model(X_train, y_train, X_val, y_val, input_dim, num_epochs=100, lr=0.01):
    model = ANNModel(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate on validation set
    with torch.no_grad():
        predicted = model(X_val).round()
        accuracy = (predicted.eq(y_val).sum() / float(y_val.shape[0])).item()
    
    return accuracy

outer_kf = KFold(n_splits=10, shuffle=True, random_state=42)
outer_accuracies = []

for outer_train_index, outer_test_index in outer_kf.split(X_remain):
    X_outer_train, X_outer_test = X_remain[outer_train_index], X_remain[outer_test_index]
    y_outer_train, y_outer_test = y_remain[outer_train_index], y_remain[outer_test_index]

    inner_kf = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_accuracies = []

    for inner_train_index, inner_val_index in inner_kf.split(X_outer_train):
        X_inner_train, X_inner_val = X_outer_train[inner_train_index], X_outer_train[inner_val_index]
        y_inner_train, y_inner_val = y_outer_train[inner_train_index], y_outer_train[inner_val_index]

        accuracy = train_and_evaluate_model(X_inner_train, y_inner_train, X_inner_val, y_inner_val, input_dim)
        inner_accuracies.append(accuracy)
        print(f'Inner Fold Accuracy: {accuracy * 100:.2f}%')

    mean_inner_accuracy = sum(inner_accuracies) / len(inner_accuracies)
    print(f'Outer Fold Mean Inner Accuracy: {mean_inner_accuracy * 100:.2f}%')

    outer_accuracy = train_and_evaluate_model(X_outer_train, y_outer_train, X_outer_test, y_outer_test, input_dim)
    outer_accuracies.append(outer_accuracy)
    print(f'Outer Fold Accuracy: {outer_accuracy * 100:.2f}%')

print(f'Mean Outer Accuracy: {sum(outer_accuracies)/len(outer_accuracies) * 100:.2f}%')

# Evaluate the final model on the holdout set
final_model = ANNModel(input_dim)
final_model_accuracy = train_and_evaluate_model(X_remain, y_remain, X_holdout, y_holdout, input_dim)
print(f'Holdout Set Accuracy: {final_model_accuracy * 100:.2f}%')


Inner Fold Accuracy: 47.11%
Inner Fold Accuracy: 56.61%
Inner Fold Accuracy: 52.89%
Inner Fold Accuracy: 54.96%
Inner Fold Accuracy: 48.35%
Inner Fold Accuracy: 47.11%
Inner Fold Accuracy: 50.21%
Inner Fold Accuracy: 44.81%
Inner Fold Accuracy: 49.38%
Inner Fold Accuracy: 49.38%
Outer Fold Mean Inner Accuracy: 50.08%
Outer Fold Accuracy: 49.81%
Inner Fold Accuracy: 48.76%
Inner Fold Accuracy: 51.65%
Inner Fold Accuracy: 53.31%
Inner Fold Accuracy: 47.52%
Inner Fold Accuracy: 48.35%
Inner Fold Accuracy: 52.89%
Inner Fold Accuracy: 47.30%
Inner Fold Accuracy: 47.72%
Inner Fold Accuracy: 45.23%
Inner Fold Accuracy: 53.94%
Outer Fold Mean Inner Accuracy: 49.67%
Outer Fold Accuracy: 48.70%
Inner Fold Accuracy: 51.24%
Inner Fold Accuracy: 48.76%
Inner Fold Accuracy: 49.59%
Inner Fold Accuracy: 51.65%
Inner Fold Accuracy: 48.35%
Inner Fold Accuracy: 50.83%
Inner Fold Accuracy: 52.70%
Inner Fold Accuracy: 51.04%
Inner Fold Accuracy: 47.30%
Inner Fold Accuracy: 45.64%
Outer Fold Mean Inner Accu